In [1]:
import pandas as pd
import numpy as np
import recordlinkage as rl
import missingno as msno
import openpyxl
from recordlinkage.datasets import load_febrl4
from recordlinkage.compare import String

In [15]:
from cleanco import prepare_terms, basename
from cleanco import cleanco
from cleanco import prepare_terms, basename


## Importando las bases de datos

In [16]:
neo=pd.read_excel('NEO. Planeación Cuentas con servicios.xlsx')

In [6]:
gaze=pd.read_excel('Base - gazelle_companies_2021-09-09_VF.xlsx')

In [7]:
fdi=pd.read_excel('Segmentación Europa FDI Markets y Preqin.xlsx')

In [6]:
gaze['Name'] = gaze['Name'].str.capitalize() ##conviertiendo en primera mayuscula resto minuscula

In [17]:
neo['Nombre ajustado'] = neo['Empresa'].apply(lambda x: cleanco(x).clean_name() if type(x)==str else x)

In [18]:
gaze['Nombre ajustado'] = gaze['Name'].apply(lambda x: cleanco(x).clean_name() if type(x)==str else x)

In [9]:
fdi['Nombre ajustado'] = fdi['Company name'].apply(lambda x: cleanco(x).clean_name() if type(x)==str else x)

In [19]:
# List of keywords to help identify stop_words
vendor_stopwords=['biz', 'bv', 'co', 'comp', 'company', 
                'corp','corporation', 'dba', 
                'inc', 'incorp', 'incorporat', 
                'incorporate', 'incorporated', 'incorporation', 
                'international', 'intl', 'intnl', 
                'limited' ,'llc', 'ltd', 'llp', 
                'machines', 'pvt', 'pte', 'private', 'unknown']

In [24]:
neo['Nombre ajustado'] = neo['Empresa'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords_neo)])) # remove stop words

AttributeError: 'int' object has no attribute 'split'

In [ ]:
# Remove vendor specific stop words
def clean_stopwords(text,eng=False):
    if eng == False:
        custom = vendor_stopwords
    else:
        custom = vendor_stopwords
    for x in custom:
        pattern2 = r'\b'+x+r'\b'
        text=re.sub(pattern2,'',text)
    return text

In [ ]:
df['x'] = df['x'].apply(lambda x: clean_stopwords(x,eng=False))

In [11]:
neo1=neo.copy()
fdi1=fdi.copy()
gaze1=gaze.copy()

In [12]:
gaze1=gaze1.rename(columns={"Country": "HQ country"}) ###Cambiando columna con el nombre de la emppresa para comparar con fdi

In [13]:
gaze1['Orden'] = gaze1.index #creando nueva columna en gazelle
fdi1['Orden'] = fdi1.index

In [ ]:
## en caso de querer agrupar por alguna categoria o bloque:
#neo1["bloque"] = "Grupo único"
#fdi1["bloque"] = "Grupo único"
#gaz11["bloque"] = "Grupo único"

In [14]:
#Haciendo un bloque con la columna en común entre las bases de datos fdi y gazelle
indexer = rl.Index()
indexer.block('HQ country')
candidate_links = indexer.index(fdi1, gaze1)

In [15]:
df_a = pd.DataFrame(fdi1)
df_b = pd.DataFrame(gaze1)

In [16]:
str(len(candidate_links))

'2760852'

In [27]:
fdi1

,Unnamed: 0,Fuente,HQ country,País empresa,Company name,Primary sector,No. of projects (in current search),Total capex USDm (in current search),Propensión a invertir según Gazelle,Exporta\nSí/No,...,No. of projects in Middle East (Overall),No. of projects in Rest of Europe (Overall),No. of projects in Western Europe (Overall),No. of projects in North America (Overall),No. of signals in current search,Date of last signal,No of contacts,Unnamed: 47,Nombre ajustado,Orden
0,0,FDI Markets,Germany,Alemania,Volkswagen,Automotive OEM,334.0,57286.10,4,NO,...,13,147,134,45,36,Nov/2020,353,NaN,Volkswagen,0
1,1,FDI Markets,Germany,Alemania,BASF,Chemicals,235.0,38636.40,-,-,...,9,38,66,48,13,Jul/2019,245,NaN,BASF,1
2,2,FDI Markets,Germany,Alemania,Deutsche Post,Transportation & Warehousing,506.0,30565.38,5,SI,...,51,97,238,124,19,Mar/2020,305,NaN,Deutsche Post,2
3,3,FDI Markets,Germany,Alemania,Daimler AG,Automotive OEM,163.0,25804.48,-,-,...,9,38,41,78,13,Feb/2019,170,NaN,Daimler,3
4,4,FDI Markets,Germany,Alemania,Siemens,Industrial equipment,283.0,19261.85,-,-,...,23,114,152,75,17,Mar/2019,284,NaN,Siemens,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4403,4403,FDI Markets,Russia,Rusia,Rimsco,Transportation & Warehousing,2.0,122.20,-,-,...,0,0,0,NaN,0,NaN,NaN,NaN,Rimsco,4403
4404,4404,FDI Markets,Russia,Rusia,Roscosmos,Space & defence,2.0,40.20,-,-,...,0,0,0,Apr/2014,2,NaN,NaN,NaN,Roscosmos,4404
4405,4405,FDI Markets,Russia,Rusia,Ruschrome Mining,Metals,2.0,1600.00,-,-,...,0,0,0,NaN,0,NaN,NaN,NaN,Ruschrome Mining,4405
4406,4406,FDI Markets,Russia,Rusia,RussNeft,"Coal, oil & gas",2.0,131.50,-,-,...,1,0,1,Nov/2011,0,NaN,NaN,NaN,RussNeft,4406


### Condiciones de comparación

In [22]:
compare = rl.Compare()
#compare.string('Company name', 'Empresa', method='levenshtein', threshold=0.7, label='Empresa')
compare.string('Nombre ajustado', 'Nombre ajustado', method='levenshtein', threshold=0.85, label='Nombre ajustado')
compare.exact('HQ country', 'HQ country', label='HQ country')
compare.string('Nombre ajustado', 'Nombre ajustado', method='jarowinkler', threshold=0.85, label='Nombre ajustado')

<Compare>

In [23]:
# The comparison vectors
compare_vectors = compare.compute(candidate_links, df_a, df_b)

In [24]:
compare_vectors

Nombre ajustado  HQ country  Nombre ajustado
0    0                 0.0           1              0.0
     10                0.0           1              0.0
     14                0.0           1              0.0
     17                0.0           1              0.0
     30                0.0           1              0.0
...                    ...         ...              ...
4209 6128              0.0           1              0.0
     7116              0.0           1              0.0
     7232              0.0           1              0.0
     7328              0.0           1              0.0
     7459              0.0           1              0.0

[2760852 rows x 3 columns]

In [26]:
# We are going to take the observations that have more than 3 criteria that passed the validation. 
matches = compare_vectors[compare_vectors.sum(axis=1) > 2]
matches

Nombre ajustado  HQ country  Nombre ajustado
0    1710              1.0           1              1.0
2    3926              1.0           1              1.0
     5993              1.0           1              1.0
     6855              1.0           1              1.0
16   1174              1.0           1              1.0
...                    ...         ...              ...
3785 4246              1.0           1              1.0
3870 3496              1.0           1              1.0
3898 4468              1.0           1              1.0
3907 7328              1.0           1              1.0
4135 2911              1.0           1              1.0

[386 rows x 3 columns]

In [31]:
matches['Total'] = matches.apply(lambda x: x.sum(), axis=1)
matches

<ipython-input-31-2cc21163078e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches['Total'] = matches.apply(lambda x: x.sum(), axis=1)


Nombre ajustado  HQ country  Nombre ajustado  Total
0    81                0.0           1              1.0    2.0
     1441              0.0           1              1.0    2.0
     1710              1.0           1              1.0    3.0
     7197              0.0           1              1.0    2.0
2    1326              0.0           1              1.0    2.0
...                    ...         ...              ...    ...
3907 7328              1.0           1              1.0    3.0
3928 4210              0.0           1              1.0    2.0
4003 4252              0.0           1              1.0    2.0
4135 2911              1.0           1              1.0    3.0
4205 3496              0.0           1              1.0    2.0

[1446 rows x 4 columns]

In [32]:
# Convert the indexes into variables to be able to do the match.
reset_df = matches.reset_index()

# Rename columns
reset_df = reset_df.rename({'level_0': 'Posición_FDI', 'level_1': 'Posición_Gazelle'}, axis=1)

In [33]:
reset_df

,Posición_FDI,Posición_Gazelle,Nombre ajustado,HQ country,Nombre ajustado,Total
0,0,81,0.0,1,1.0,2.0
1,0,1441,0.0,1,1.0,2.0
2,0,1710,1.0,1,1.0,3.0
3,0,7197,0.0,1,1.0,2.0
4,2,1326,0.0,1,1.0,2.0
...,...,...,...,...,...,...
1441,3907,7328,1.0,1,1.0,3.0
1442,3928,4210,0.0,1,1.0,2.0
1443,4003,4252,0.0,1,1.0,2.0
1444,4135,2911,1.0,1,1.0,3.0


In [ ]:
# Copy
df_merge = fdi1.copy()
df_merge.shape


In [37]:
# Make a merge to understand which one there is a match with.
df_merge = pd.merge( fdi1 , reset_df[['Posición_FDI', 'Posición_Gazelle', 'Total' ]] , how = 'left' , 
              left_on = [ 'Orden' ] , right_on = [ 'Posición_FDI' ])
df_merge

,Unnamed: 0,Fuente,HQ country,País empresa,Company name,Primary sector,No. of projects (in current search),Total capex USDm (in current search),Propensión a invertir según Gazelle,Exporta\nSí/No,...,No. of projects in North America (Overall),No. of signals in current search,Date of last signal,No of contacts,Unnamed: 47,Nombre ajustado,Orden,Posición_FDI,Posición_Gazelle,Total
0,0,FDI Markets,Germany,Alemania,Volkswagen,Automotive OEM,334.0,57286.1,4,NaN,...,45,36,Nov/2020,353,NaN,Volkswagen,0,0.0,81.0,2.0
1,0,FDI Markets,Germany,Alemania,Volkswagen,Automotive OEM,334.0,57286.1,4,NaN,...,45,36,Nov/2020,353,NaN,Volkswagen,0,0.0,1441.0,2.0
2,0,FDI Markets,Germany,Alemania,Volkswagen,Automotive OEM,334.0,57286.1,4,NaN,...,45,36,Nov/2020,353,NaN,Volkswagen,0,0.0,1710.0,3.0
3,0,FDI Markets,Germany,Alemania,Volkswagen,Automotive OEM,334.0,57286.1,4,NaN,...,45,36,Nov/2020,353,NaN,Volkswagen,0,0.0,7197.0,2.0
4,1,FDI Markets,Germany,Alemania,BASF,Chemicals,235.0,38636.4,5,NaN,...,48,13,Jul/2019,245,NaN,BASF,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5080,4403,FDI Markets,Russia,Rusia,Rimsco,Transportation & Warehousing,2.0,122.2,5,NaN,...,NaN,0,NaN,NaN,NaN,Rimsco,4403,NaN,NaN,NaN
5081,4404,FDI Markets,Russia,Rusia,Roscosmos,Space & defence,2.0,40.2,5,NaN,...,Apr/2014,2,NaN,NaN,NaN,Roscosmos,4404,NaN,NaN,NaN
5082,4405,FDI Markets,Russia,Rusia,Ruschrome Mining,Metals,2.0,1600.0,5,NaN,...,NaN,0,NaN,NaN,NaN,Ruschrome Mining,4405,NaN,NaN,NaN
5083,4406,FDI Markets,Russia,Rusia,RussNeft,"Coal, oil & gas",2.0,131.5,5,NaN,...,Nov/2011,0,NaN,NaN,NaN,RussNeft,4406,NaN,NaN,NaN


In [39]:
# Unique Record Count
df_merge["Total"].isna().sum()
df_merge

,Unnamed: 0,Fuente,HQ country,País empresa,Company name,Primary sector,No. of projects (in current search),Total capex USDm (in current search),Propensión a invertir según Gazelle,Exporta\nSí/No,...,No. of projects in North America (Overall),No. of signals in current search,Date of last signal,No of contacts,Unnamed: 47,Nombre ajustado,Orden,Posición_FDI,Posición_Gazelle,Total
0,0,FDI Markets,Germany,Alemania,Volkswagen,Automotive OEM,334.0,57286.1,4,NaN,...,45,36,Nov/2020,353,NaN,Volkswagen,0,0.0,81.0,2.0
1,0,FDI Markets,Germany,Alemania,Volkswagen,Automotive OEM,334.0,57286.1,4,NaN,...,45,36,Nov/2020,353,NaN,Volkswagen,0,0.0,1441.0,2.0
2,0,FDI Markets,Germany,Alemania,Volkswagen,Automotive OEM,334.0,57286.1,4,NaN,...,45,36,Nov/2020,353,NaN,Volkswagen,0,0.0,1710.0,3.0
3,0,FDI Markets,Germany,Alemania,Volkswagen,Automotive OEM,334.0,57286.1,4,NaN,...,45,36,Nov/2020,353,NaN,Volkswagen,0,0.0,7197.0,2.0
4,1,FDI Markets,Germany,Alemania,BASF,Chemicals,235.0,38636.4,5,NaN,...,48,13,Jul/2019,245,NaN,BASF,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5080,4403,FDI Markets,Russia,Rusia,Rimsco,Transportation & Warehousing,2.0,122.2,5,NaN,...,NaN,0,NaN,NaN,NaN,Rimsco,4403,NaN,NaN,NaN
5081,4404,FDI Markets,Russia,Rusia,Roscosmos,Space & defence,2.0,40.2,5,NaN,...,Apr/2014,2,NaN,NaN,NaN,Roscosmos,4404,NaN,NaN,NaN
5082,4405,FDI Markets,Russia,Rusia,Ruschrome Mining,Metals,2.0,1600.0,5,NaN,...,NaN,0,NaN,NaN,NaN,Ruschrome Mining,4405,NaN,NaN,NaN
5083,4406,FDI Markets,Russia,Rusia,RussNeft,"Coal, oil & gas",2.0,131.5,5,NaN,...,Nov/2011,0,NaN,NaN,NaN,RussNeft,4406,NaN,NaN,NaN


In [40]:
df_merge["Total"].isna().sum()

df_valores_unidos = df_merge[df_merge['Total'].isnull()]

In [41]:
df_valores_unidos

,Unnamed: 0,Fuente,HQ country,País empresa,Company name,Primary sector,No. of projects (in current search),Total capex USDm (in current search),Propensión a invertir según Gazelle,Exporta\nSí/No,...,No. of projects in North America (Overall),No. of signals in current search,Date of last signal,No of contacts,Unnamed: 47,Nombre ajustado,Orden,Posición_FDI,Posición_Gazelle,Total
4,1,FDI Markets,Germany,Alemania,BASF,Chemicals,235.0,38636.40,5,NaN,...,48,13,Jul/2019,245,NaN,BASF,1,NaN,NaN,NaN
16,3,FDI Markets,Germany,Alemania,Daimler AG,Automotive OEM,163.0,25804.48,3,NaN,...,78,13,Feb/2019,170,NaN,Daimler,3,NaN,NaN,NaN
28,8,FDI Markets,Germany,Alemania,SAIC Volkswagen (Shanghai Volkswagen Automotive),Automotive OEM,8.0,11966.12,4,NaN,...,0,0,NaN,0,NaN,SAIC Volkswagen (Shanghai Volkswagen Automotive,8,NaN,NaN,NaN
42,11,FDI Markets,Germany,Alemania,DaimlerChrysler,Automotive OEM,51.0,9324.09,3,NaN,...,27,0,NaN,10,NaN,DaimlerChrysler,11,NaN,NaN,NaN
44,13,FDI Markets,Germany,Alemania,DHL Express,Transportation & Warehousing,175.0,8508.15,5,NaN,...,22,7,Nov/2018,0,NaN,DHL Express,13,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5080,4403,FDI Markets,Russia,Rusia,Rimsco,Transportation & Warehousing,2.0,122.20,5,NaN,...,NaN,0,NaN,NaN,NaN,Rimsco,4403,NaN,NaN,NaN
5081,4404,FDI Markets,Russia,Rusia,Roscosmos,Space & defence,2.0,40.20,5,NaN,...,Apr/2014,2,NaN,NaN,NaN,Roscosmos,4404,NaN,NaN,NaN
5082,4405,FDI Markets,Russia,Rusia,Ruschrome Mining,Metals,2.0,1600.00,5,NaN,...,NaN,0,NaN,NaN,NaN,Ruschrome Mining,4405,NaN,NaN,NaN
5083,4406,FDI Markets,Russia,Rusia,RussNeft,"Coal, oil & gas",2.0,131.50,5,NaN,...,Nov/2011,0,NaN,NaN,NaN,RussNeft,4406,NaN,NaN,NaN


In [42]:
df_merge

,Unnamed: 0,Fuente,HQ country,País empresa,Company name,Primary sector,No. of projects (in current search),Total capex USDm (in current search),Propensión a invertir según Gazelle,Exporta\nSí/No,...,No. of projects in North America (Overall),No. of signals in current search,Date of last signal,No of contacts,Unnamed: 47,Nombre ajustado,Orden,Posición_FDI,Posición_Gazelle,Total
0,0,FDI Markets,Germany,Alemania,Volkswagen,Automotive OEM,334.0,57286.1,4,NaN,...,45,36,Nov/2020,353,NaN,Volkswagen,0,0.0,81.0,2.0
1,0,FDI Markets,Germany,Alemania,Volkswagen,Automotive OEM,334.0,57286.1,4,NaN,...,45,36,Nov/2020,353,NaN,Volkswagen,0,0.0,1441.0,2.0
2,0,FDI Markets,Germany,Alemania,Volkswagen,Automotive OEM,334.0,57286.1,4,NaN,...,45,36,Nov/2020,353,NaN,Volkswagen,0,0.0,1710.0,3.0
3,0,FDI Markets,Germany,Alemania,Volkswagen,Automotive OEM,334.0,57286.1,4,NaN,...,45,36,Nov/2020,353,NaN,Volkswagen,0,0.0,7197.0,2.0
4,1,FDI Markets,Germany,Alemania,BASF,Chemicals,235.0,38636.4,5,NaN,...,48,13,Jul/2019,245,NaN,BASF,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5080,4403,FDI Markets,Russia,Rusia,Rimsco,Transportation & Warehousing,2.0,122.2,5,NaN,...,NaN,0,NaN,NaN,NaN,Rimsco,4403,NaN,NaN,NaN
5081,4404,FDI Markets,Russia,Rusia,Roscosmos,Space & defence,2.0,40.2,5,NaN,...,Apr/2014,2,NaN,NaN,NaN,Roscosmos,4404,NaN,NaN,NaN
5082,4405,FDI Markets,Russia,Rusia,Ruschrome Mining,Metals,2.0,1600.0,5,NaN,...,NaN,0,NaN,NaN,NaN,Ruschrome Mining,4405,NaN,NaN,NaN
5083,4406,FDI Markets,Russia,Rusia,RussNeft,"Coal, oil & gas",2.0,131.5,5,NaN,...,Nov/2011,0,NaN,NaN,NaN,RussNeft,4406,NaN,NaN,NaN


In [43]:
data = df_merge.dropna(subset=["Posición_Gazelle"])

In [44]:
data
data.to_excel("fdi-gaz.xlsx")

,Unnamed: 0,Fuente,HQ country,País empresa,Company name,Primary sector,No. of projects (in current search),Total capex USDm (in current search),Propensión a invertir según Gazelle,Exporta\nSí/No,...,No. of projects in North America (Overall),No. of signals in current search,Date of last signal,No of contacts,Unnamed: 47,Nombre ajustado,Orden,Posición_FDI,Posición_Gazelle,Total
0,0,FDI Markets,Germany,Alemania,Volkswagen,Automotive OEM,334.0,57286.10,4,NaN,...,45,36,Nov/2020,353,NaN,Volkswagen,0,0.0,81.0,2.0
1,0,FDI Markets,Germany,Alemania,Volkswagen,Automotive OEM,334.0,57286.10,4,NaN,...,45,36,Nov/2020,353,NaN,Volkswagen,0,0.0,1441.0,2.0
2,0,FDI Markets,Germany,Alemania,Volkswagen,Automotive OEM,334.0,57286.10,4,NaN,...,45,36,Nov/2020,353,NaN,Volkswagen,0,0.0,1710.0,3.0
3,0,FDI Markets,Germany,Alemania,Volkswagen,Automotive OEM,334.0,57286.10,4,NaN,...,45,36,Nov/2020,353,NaN,Volkswagen,0,0.0,7197.0,2.0
5,2,FDI Markets,Germany,Alemania,Deutsche Post,Transportation & Warehousing,506.0,30565.38,3,NaN,...,124,19,Mar/2020,305,NaN,Deutsche Post,2,2.0,1326.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4584,3907,FDI Markets,Turkey,Turquía,Enka Insaat ve Sanayi,Food & Beverages,2.0,71.90,5,NaN,...,0,NaN,1,NaN,NaN,Enka Insaat ve Sanayi,3907,3907.0,7328.0,3.0
4605,3928,FDI Markets,Turkey,Turquía,Turkiye Vakiflar Bankasi,Financial services,2.0,30.20,5,NaN,...,0,NaN,1,NaN,NaN,Turkiye Vakiflar Bankasi,3928,3928.0,4210.0,2.0
4680,4003,FDI Markets,Turkey,Turquía,Cimsa Cimento,Building materials,1.0,0.40,5,NaN,...,0,Oct/2010,2,NaN,NaN,Cimsa Cimento,4003,4003.0,4252.0,2.0
4812,4135,FDI Markets,Turkey,Turquía,Opet,"Coal, oil & gas",1.0,6.50,5,NaN,...,0,NaN,0,NaN,NaN,Opet,4135,4135.0,2911.0,3.0


## FDI y NEO

In [61]:
neo=pd.read_excel('NEO. Planeación Cuentas con servicios.xlsx')

In [62]:
fdi=pd.read_excel('Segmentación Europa FDI Markets y Preqin.xlsx')

In [81]:
neo['Nombre ajustado'] = neo['Empresa'].apply(lambda x: cleanco(x).clean_name() if type(x)==str else x)

In [64]:
fdi['Nombre ajustado'] = fdi['Company name'].apply(lambda x: cleanco(x).clean_name() if type(x)==str else x)

In [96]:
neo1=neo.copy()
fdi1=fdi.copy()

In [ ]:
#neo1=gaze1.rename(columns={"Country": "HQ country"}) ###Cambiando columna con el nombre de la emppresa para comparar con fdi

In [97]:
neo1['Orden'] = neo1.index #creando nueva columna en gazelle
fdi1['Orden'] = fdi1.index

In [98]:
#Haciendo un bloque con la columna en común entre las bases de datos fdi y gazelle
indexer = rl.Index()
indexer.block('País empresa')
candidate_links = indexer.index(fdi1, neo1)

In [99]:
df_a = pd.DataFrame(fdi1)
df_b = pd.DataFrame(neo1)

In [90]:
dfc=df_a.copy()

In [92]:
neo['Nombre ajustado'] = neo['Empresa'].apply(lambda x: cleanco(str(x)).clean_name())

In [93]:
str(len(candidate_links))

'1839022'

In [100]:
compare = rl.Compare()
#compare.string('Company name', 'Empresa', method='levenshtein', threshold=0.7, label='Empresa')
compare.string('Nombre ajustado', 'Nombre ajustado', method='levenshtein', threshold=0.85, label='Nombre ajustado')
compare.exact('País empresa', 'País empresa', label='País empresa')
compare.string('Nombre ajustado', 'Nombre ajustado', method='jarowinkler', threshold=0.85, label='Nombre ajustado')

<Compare>

In [101]:
# The comparison vectors
compare_vectors = compare.compute(candidate_links, df_a, df_b)

In [104]:
# We are going to take the observations that have more than 3 criteria that passed the validation. 
matches = compare_vectors[compare_vectors.sum(axis=1) > 1]
matches

Nombre ajustado  País empresa  Nombre ajustado
0    22165              1.0             1              1.0
     22166              1.0             1              1.0
1    2307               1.0             1              1.0
2    5778               0.0             1              1.0
     5779               0.0             1              1.0
...                     ...           ...              ...
4295 7946               1.0             1              1.0
4307 21443              1.0             1              1.0
     21444              1.0             1              1.0
     21445              1.0             1              1.0
     21446              1.0             1              1.0

[2176 rows x 3 columns]

In [105]:
matches['Total'] = matches.apply(lambda x: x.sum(), axis=1)
matches

<ipython-input-105-2cc21163078e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches['Total'] = matches.apply(lambda x: x.sum(), axis=1)


Nombre ajustado  País empresa  Nombre ajustado  Total
0    22165              1.0             1              1.0    3.0
     22166              1.0             1              1.0    3.0
1    2307               1.0             1              1.0    3.0
2    5778               0.0             1              1.0    2.0
     5779               0.0             1              1.0    2.0
...                     ...           ...              ...    ...
4295 7946               1.0             1              1.0    3.0
4307 21443              1.0             1              1.0    3.0
     21444              1.0             1              1.0    3.0
     21445              1.0             1              1.0    3.0
     21446              1.0             1              1.0    3.0

[2176 rows x 4 columns]

In [110]:
# Convert the indexes into variables to be able to do the match.
reset_df = matches.reset_index()

# Rename columns
reset_df = reset_df.rename({'level_0': 'Posición_FDI', 'level_1': 'Posición_neo'}, axis=1)

In [111]:
reset_df

,Posición_FDI,Posición_neo,Nombre ajustado,País empresa,Nombre ajustado,Total
0,0,22165,1.0,1,1.0,3.0
1,0,22166,1.0,1,1.0,3.0
2,1,2307,1.0,1,1.0,3.0
3,2,5778,0.0,1,1.0,2.0
4,2,5779,0.0,1,1.0,2.0
...,...,...,...,...,...,...
2171,4295,7946,1.0,1,1.0,3.0
2172,4307,21443,1.0,1,1.0,3.0
2173,4307,21444,1.0,1,1.0,3.0
2174,4307,21445,1.0,1,1.0,3.0


In [112]:
# Copy
df_merge = neo1.copy()
df_merge.shape

(23146, 28)

In [113]:
df_merge

,Unnamed: 0,Empresa,Tipo empresa,País empresa,Instrumento o tarea,Tipo registro,Descripción,Cadena productiva,Sector,Estado del servicio,...,Tipo proyecto,Fecha realizacion proyecto,Empresa Ancla,Trimestre,Nombre cuenta Principal,Pais cuenta Principal,Inversionista instalado,Link del servicio a NEO,Nombre ajustado,Orden
0,0,1871,Exterior,Estados Unidos,Instrumento,AGENDA DE INVERSIÓN,Participacion agenda de inversion de IIB en At...,Industrias 4.0,Software y Servicios TI,Finalizado,...,AGENDA DE INVERSIÓN,NaT,No,Q2,NaN,NaN,No,https://procolombia.lightning.force.com/a0Z0y0...,1871,0
1,1,1 DOC 3,Nacional,Colombia,Instrumento,AGENDA DE INVERSIÓN,Participación Agenda de Inversion - Future Per...,Industrias 4.0,Publicidad y mercadeo digital,Finalizado,...,AGENDA DE INVERSIÓN,NaT,No,Q3,NaN,NaN,No,https://procolombia.lightning.force.com/a0Z0y0...,1 DOC 3,1
2,2,1000 Volt Post Production,Exterior,Turquía,Instrumento,PRIMERA VISITA,Primera Visita 1000 Volt Production,Otros,Otros servicios,Finalizado,...,NO DEFINIDO,NaT,No,Q3,NaN,NaN,No,https://procolombia.lightning.force.com/a0Z0y0...,1000 Volt Post Production,2
3,3,10Adventures,Exterior,Canadá,Instrumento,PRIMERA VISITA,Primera Visita 10Adventures,Industrias 4.0,Software y Servicios TI,Finalizado,...,NO DEFINIDO,NaT,No,Q4,NaN,NaN,No,https://procolombia.lightning.force.com/a0Z4X0...,10Adventures,3
4,4,1291 Group,Exterior,Alemania,Instrumento,PRIMERA VISITA,Participación en desayuno de inversión con Pre...,Otros,Servicios financieros y aseguradoras,Finalizado,...,EVENTOS DE INVERSIÓN,2018-05-18,No,Q2,NaN,NaN,No,https://procolombia.lightning.force.com/a0Z0y0...,1291 Group,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23141,23141,Zurich International School,Exterior,Suiza,Instrumento,PRIMERA VISITA,Primera visita,Industrias 4.0,Educación y consultoría gerencial,Seleccionado,...,EVENTOS DE INVERSIÓN,2019-09-18,No,Q4,NaN,NaN,No,https://procolombia.lightning.force.com/a0Z0y0...,Zurich International School,23141
23142,23142,ZYFRA,Exterior,Rusia,Instrumento,PRIMERA VISITA,Primera Visita. Zyfra,Otros,Otros servicios,Finalizado,...,NO DEFINIDO,NaT,No,Q3,NaN,NaN,No,https://procolombia.lightning.force.com/a0Z0y0...,ZYFRA,23142
23143,23143,ZYFRA,Exterior,Rusia,Instrumento,AGENDA DE INVERSIÓN,Agenda Bgta- Minería,Otros,Otros servicios,Finalizado,...,AGENDA DE INVERSIÓN,2019-09-09,No,Q3,NaN,NaN,No,https://procolombia.lightning.force.com/a0Z0y0...,ZYFRA,23143
23144,23144,ZYNCH,Exterior,México,Instrumento,PRIMERA VISITA,Primera Visita ZYNCH,Metalmecánica y otras industrias,Vehículos y otros medios de transporte,Finalizado,...,NO DEFINIDO,NaT,No,Q1,NaN,NaN,No,https://procolombia.lightning.force.com/a0Z0y0...,ZYNCH,23144


In [114]:
reset_df
reset_df.to_excel("neo_fdi.xlsx")

## prequin y neo

In [30]:
preq =pd.read_excel('Segmentación Europa FDI Markets y Preqin.xlsx', 'Preqin')

In [38]:
preq

,Fuente,HQ country,País empresa,Company name,Primary sector,No. of projects (in current search),Total capex USDm (in current search),Propensión a invertir según Gazelle,Exporta\nSí/No,De Preqin \n# Deals en Latam,...,No. of projects in Asia-pacific (Overall),No. of projects in Latin america (Overall),No. of projects in Middle East (Overall),No. of projects in Rest of Europe (Overall),No. of projects in Western Europe (Overall),No. of projects in North America (Overall),No. of signals in current search,Date of last signal,No of contacts,Nombre ajustado
0,Preqin,Germany,Alemania,Global Founders Capital,Fund Manager,NaN,NaN,NaN,NaN,22.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Global Founders Capital
1,Preqin,Germany,Alemania,Rocket Internet SE,Fund Manager,NaN,NaN,NaN,NaN,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rocket Internet
2,Preqin,Switzerland,Suiza,Partners Group,Fund Manager,NaN,NaN,NaN,NaN,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Partners Group
3,Preqin,Switzerland,Suiza,Vital Capital Investments,Fund Manager,NaN,NaN,NaN,NaN,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vital Capital Investments
4,Preqin,Germany,Alemania,TEV Team Management GmbH,Fund Manager,NaN,NaN,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TEV Team Management
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,Preqin,Russia,Rusia,RUSNANO Management Company,Fund Manager,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUSNANO Management
272,Preqin,Russia,Rusia,Primer Capital,Fund Manager,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Primer Capital
273,Preqin,Russia,Rusia,SUN Group,Fund Manager,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SUN Group
274,Preqin,Russia,Rusia,VTB Capital,Fund Manager,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,VTB Capital


In [37]:
neo

,Empresa,Tipo empresa,País empresa,Instrumento o tarea,Tipo registro,Descripción,Cadena productiva,Sector,Estado del servicio,Fecha creación del servicio,...,Fecha inicio proyecto,Tipo proyecto,Fecha realizacion proyecto,Empresa Ancla,Trimestre,Nombre cuenta Principal,Pais cuenta Principal,Inversionista instalado,Link del servicio a NEO,Nombre ajustado
0,1871,Exterior,Estados Unidos,Instrumento,AGENDA DE INVERSIÓN,Participacion agenda de inversion de IIB en At...,Industrias 4.0,Software y Servicios TI,Finalizado,2019-06-13,...,2019-05-13,AGENDA DE INVERSIÓN,NaT,No,Q2,NaN,NaN,No,https://procolombia.lightning.force.com/a0Z0y0...,1871
1,1 DOC 3,Nacional,Colombia,Instrumento,AGENDA DE INVERSIÓN,Participación Agenda de Inversion - Future Per...,Industrias 4.0,Publicidad y mercadeo digital,Finalizado,2020-08-20,...,2020-07-15,AGENDA DE INVERSIÓN,NaT,No,Q3,NaN,NaN,No,https://procolombia.lightning.force.com/a0Z0y0...,1 DOC 3
2,1000 Volt Post Production,Exterior,Turquía,Instrumento,PRIMERA VISITA,Primera Visita 1000 Volt Production,Otros,Otros servicios,Finalizado,2020-08-18,...,NaT,NO DEFINIDO,NaT,No,Q3,NaN,NaN,No,https://procolombia.lightning.force.com/a0Z0y0...,1000 Volt Post Production
3,10Adventures,Exterior,Canadá,Instrumento,PRIMERA VISITA,Primera Visita 10Adventures,Industrias 4.0,Software y Servicios TI,Finalizado,2020-12-08,...,NaT,NO DEFINIDO,NaT,No,Q4,NaN,NaN,No,https://procolombia.lightning.force.com/a0Z4X0...,10Adventures
4,1291 Group,Exterior,Alemania,Instrumento,PRIMERA VISITA,Participación en desayuno de inversión con Pre...,Otros,Servicios financieros y aseguradoras,Finalizado,2018-06-20,...,2018-04-25,EVENTOS DE INVERSIÓN,2018-05-18,No,Q2,NaN,NaN,No,https://procolombia.lightning.force.com/a0Z0y0...,1291 Group
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23141,Zurich International School,Exterior,Suiza,Instrumento,PRIMERA VISITA,Primera visita,Industrias 4.0,Educación y consultoría gerencial,Seleccionado,2019-10-10,...,2019-07-04,EVENTOS DE INVERSIÓN,2019-09-18,No,Q4,NaN,NaN,No,https://procolombia.lightning.force.com/a0Z0y0...,Zurich International School
23142,ZYFRA,Exterior,Rusia,Instrumento,PRIMERA VISITA,Primera Visita. Zyfra,Otros,Otros servicios,Finalizado,2019-08-13,...,NaT,NO DEFINIDO,NaT,No,Q3,NaN,NaN,No,https://procolombia.lightning.force.com/a0Z0y0...,ZYFRA
23143,ZYFRA,Exterior,Rusia,Instrumento,AGENDA DE INVERSIÓN,Agenda Bgta- Minería,Otros,Otros servicios,Finalizado,2019-09-09,...,2019-09-09,AGENDA DE INVERSIÓN,2019-09-09,No,Q3,NaN,NaN,No,https://procolombia.lightning.force.com/a0Z0y0...,ZYFRA
23144,ZYNCH,Exterior,México,Instrumento,PRIMERA VISITA,Primera Visita ZYNCH,Metalmecánica y otras industrias,Vehículos y otros medios de transporte,Finalizado,2020-02-19,...,NaT,NO DEFINIDO,NaT,No,Q1,NaN,NaN,No,https://procolombia.lightning.force.com/a0Z0y0...,ZYNCH


In [32]:
neo['Nombre ajustado'] = neo['Empresa'].apply(lambda x: cleanco(x).clean_name() if type(x)==str else x)

In [35]:
preq['Nombre ajustado'] = preq['Company name'].apply(lambda x: cleanco(x).clean_name() if type(x)==str else x)

In [36]:
neo1=neo.copy()
preq1=preq.copy()

In [39]:
neo1['Orden'] = neo1.index #creando nueva columna en gazelle
preq1['Orden'] = preq1.index

In [40]:
#Haciendo un bloque con la columna en común entre las bases de datos fdi y gazelle
indexer = rl.Index()
indexer.block('País empresa')
candidate_links = indexer.index(neo1, preq1)

In [41]:
df_a = pd.DataFrame(neo1)
df_b = pd.DataFrame(preq1)

In [43]:
str(len(candidate_links))

'143402'

In [44]:
compare = rl.Compare()
#compare.string('Company name', 'Empresa', method='levenshtein', threshold=0.7, label='Empresa')
compare.string('Nombre ajustado', 'Nombre ajustado', method='levenshtein', threshold=0.85, label='Nombre ajustado')
compare.exact('País empresa', 'País empresa', label='HQ country')
compare.string('Nombre ajustado', 'Nombre ajustado', method='jarowinkler', threshold=0.85, label='Nombre ajustado')

<Compare>

In [45]:
# The comparison vectors
compare_vectors = compare.compute(candidate_links, df_a, df_b)

In [46]:
compare_vectors

Nombre ajustado  HQ country  Nombre ajustado
2     264              0.0           1              0.0
      265              0.0           1              0.0
      266              0.0           1              0.0
375   264              0.0           1              0.0
      265              0.0           1              0.0
...                    ...         ...              ...
21028 223              0.0           1              0.0
      233              0.0           1              0.0
22250 256              0.0           1              0.0
      259              0.0           1              0.0
      260              0.0           1              0.0

[143402 rows x 3 columns]

In [48]:
# We are going to take the observations that have more than 3 criteria that passed the validation. 
matches = compare_vectors[compare_vectors.sum(axis=1) > 1]
matches

,,Nombre ajustado,HQ country,Nombre ajustado
899,47,1.0,1,1.0
900,47,1.0,1,1.0
355,107,1.0,1,1.0
356,107,1.0,1,1.0
357,107,1.0,1,1.0
...,...,...,...,...
16139,2,1.0,1,1.0
16140,2,1.0,1,1.0
16141,2,1.0,1,1.0
16142,2,1.0,1,1.0


In [56]:
matches['Total'] = matches.apply(lambda x: x.sum(), axis=1)
matches

<ipython-input-56-2cc21163078e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches['Total'] = matches.apply(lambda x: x.sum(), axis=1)


,,Nombre ajustado,HQ country,Nombre ajustado,Total
899,47,1.0,1,1.0,6.0
900,47,1.0,1,1.0,6.0
355,107,1.0,1,1.0,6.0
356,107,1.0,1,1.0,6.0
357,107,1.0,1,1.0,6.0
...,...,...,...,...,...
16139,2,1.0,1,1.0,6.0
16140,2,1.0,1,1.0,6.0
16141,2,1.0,1,1.0,6.0
16142,2,1.0,1,1.0,6.0


In [57]:
# Convert the indexes into variables to be able to do the match.
reset_df = matches.reset_index()

# Rename columns
reset_df = reset_df.rename({'level_0': 'Posición_neo', 'level_1': 'Posición_preqin'}, axis=1)

In [58]:
reset_df

,Posición_neo,Posición_preqin,Nombre ajustado,HQ country,Nombre ajustado,Total
0,899,47,1.0,1,1.0,6.0
1,900,47,1.0,1,1.0,6.0
2,355,107,1.0,1,1.0,6.0
3,356,107,1.0,1,1.0,6.0
4,357,107,1.0,1,1.0,6.0
...,...,...,...,...,...,...
96,16139,2,1.0,1,1.0,6.0
97,16140,2,1.0,1,1.0,6.0
98,16141,2,1.0,1,1.0,6.0
99,16142,2,1.0,1,1.0,6.0


In [53]:
# Copy
df_merge = preq1.copy()
df_merge.shape

(276, 48)

In [60]:
reset_df
reset_df.to_excel("preqin.xlsx")